问题描述：GAN实现MNIST手写数字识别

In [1]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


1、定义输入占位符和辅助函数

In [4]:
batch_size = 100

X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1], name='X')
noise = tf.placeholder(dtype=tf.float32, shape=[None,100], name='noise')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')

def lrelu(x, leak=0.2):
    return tf.maximum(x, leak * x)

def cross_entropy(x, y):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, labels=y)

2、定义判别器网络

In [5]:
def discriminator(image, reuse=None, is_training=is_training):
    # 这里判别器的参数要定义reuse，因为 判别器的loss=真图片loss+假图片loss，要两次用到这些参数。
    with tf.variable_scope('discriminator', reuse=reuse):
        h0 = lrelu(tf.layers.conv2d(image, kernel_size=5, filters=64, strides=2, padding='same')) #14x14x64
        
        h1 = tf.layers.conv2d(h0, kernel_size=5, filters=128, strides=2, padding='same')          #7x7x128
        h1 = lrelu(tf.contrib.layers.batch_norm(h1, is_training=is_training, decay=0.9))
        
        h2 = tf.layers.conv2d(h1, kernel_size=5, filters=256, strides=2, padding='same')          #4x4x256
        h2 = lrelu(tf.contrib.layers.batch_norm(h2, is_training=is_training, decay=0.9))
        
        h3 = tf.layers.conv2d(h2, kernel_size=5, filters=512, strides=2, padding='same')          #2x2x512
        h3 = lrelu(tf.contrib.layers.batch_norm(h3, is_training=is_training, decay=0.9))
        
        h4 = tf.contrib.layers.flatten(h3)         #2048
        h4 = tf.layers.dense(h4, units=1)
        return tf.nn.sigmoid(h4), h4

3、定义生成器网络

In [6]:
def generator(z, is_training=is_training):
    with tf.variable_scope('generator', reuse=None):
        h0 = tf.layers.dense(z, units=3*3*512) #100维输出3x3x512=4608维
        h0 = tf.reshape(h0, shape=[-1, 3, 3, 512])
        h0 = tf.nn.relu(tf.contrib.layers.batch_norm(h0, is_training=is_training, decay=0.9))      #3x3x512
        
        h1 = tf.layers.conv2d_transpose(h0, kernel_size=5, filters=256, strides=2, padding='same') #6x6x256
        h1 = tf.nn.relu(tf.contrib.layers.batch_norm(h1, is_training=is_training, decay=0.9))
        
        h2 = tf.layers.conv2d_transpose(h1, kernel_size=5, filters=128, strides=2, padding='same') #12x12x128
        h2 = tf.nn.relu(tf.contrib.layers.batch_norm(h2, is_training=is_training, decay=0.9))
        
        h3 = tf.layers.conv2d_transpose(h2, kernel_size=5, filters=64, strides=2, padding='same')  #24x24x64
        h3 = tf.nn.relu(tf.contrib.layers.batch_norm(h3, is_training=is_training, decay=0.9))
        
        h4 = tf.layers.conv2d_transpose(h3, kernel_size=5, filters=1, strides=1, padding='valid', activation=tf.nn.tanh, name='g')  #28x28x1
        
        return h4

4、定义损失函数

In [7]:
g = generator(noise)
# d_real_logits是判别器网络的最终输出值
# d_real是d_real_logits送入sigmoid激活函数之后的预测图片为真的概率
d_real, d_real_logits = discriminator(X)
d_fake, d_fake_logits = discriminator(g, reuse=True)

#把生成器网络和辨别器网络中的参数分别取出，便于后边反向传播时分别迭代
vars_g = [var for var in tf.trainable_variables() if var.name.startswith('generator')]
vars_d = [var for var in tf.trainable_variables() if var.name.startswith('discriminator')]

loss_d_real = tf.reduce_mean(cross_entropy(d_real_logits, tf.ones_like(d_real)))
loss_d_fake = tf.reduce_mean(cross_entropy(d_fake_logits, tf.zeros_like(d_fake)))
loss_g = tf.reduce_mean(cross_entropy(d_fake_logits, tf.ones_like(d_fake)))
loss_d = loss_d_real + loss_d_fake

5、定义优化器

这里我理解tf.GraphKeys.UPDATE_OPS中存放着许多tensorflow自定义的辅助函数，包括计算均值、方差等。
这里主要是为了给batch_norm层提供计算中要用到的均值方差等。
tf.control_dependencies函数是建立依赖关系，每次训练网络之前都要先获取当前输入batch的均值和方差。

In [8]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer_d = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5).minimize(loss_d, var_list=vars_d) #这里训练生成器就只传生成器参数
    optimizer_g = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5).minimize(loss_g, var_list=vars_g) 

6、执行训练

In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())


for i in range(60):
    n = np.random.uniform(-1.0, 1.0, [batch_size, 100]).astype(np.float32)
    batch = mnist.train.next_batch(batch_size=batch_size)[0]
    batch = np.reshape(batch, [-1, 28, 28, 1])
    batch = (batch - 0.5) * 2
    
    d_ls, g_ls = sess.run([loss_d, loss_g], feed_dict={X: batch, noise: n, is_training: True})
    
    sess.run(optimizer_d, feed_dict={X: batch, noise: n, is_training: True})
    sess.run(optimizer_g, feed_dict={X: batch, noise: n, is_training: True})
    sess.run(optimizer_g, feed_dict={X: batch, noise: n, is_training: True})
        
    if i % 5 == 0:
        print("step:",i," d_loss:%.4f  g_loss:%.4f"%(d_ls, g_ls))

step: 0  d_loss:1.4553  g_loss:1.2413
step: 5  d_loss:4.7823  g_loss:0.0192
step: 10  d_loss:0.4610  g_loss:7.2861
step: 15  d_loss:0.7915  g_loss:6.4483
step: 20  d_loss:0.4390  g_loss:1.5530
step: 25  d_loss:2.6375  g_loss:7.4860
step: 30  d_loss:0.8508  g_loss:0.6759
step: 35  d_loss:0.4464  g_loss:3.4547
step: 40  d_loss:1.4343  g_loss:5.5480
step: 45  d_loss:3.2633  g_loss:0.0764
step: 50  d_loss:1.3902  g_loss:0.5683
step: 55  d_loss:1.0607  g_loss:0.9524


In [ ]:
saver = tf.train.Saver()
saver.save(sess, './mnist_dcgan', global_step=60000)